In [1]:
import jCMIP as jc
import numpy as np
import glob
import os
from netCDF4 import Dataset
import matplotlib.pyplot as plt

Clist = jc.readList('../CMIP6list')

In [2]:
# function to compute means:
def compute_mean_2D(inds):
    data = np.zeros((nj,ni),'float')
    ncid = Dataset(fieldfile,'r')
    cc = 0
    for ii in inds:
        ind = np.squeeze(np.where(years_ts[ii] == years))
        if np.size(ind) == 1:
            data = data + ncid.variables[var_field][ind,:,:]
            cc = cc + 1
    ncid.close()
    
    data = data/cc
    
    return data

In [11]:
# Details for computation:
EXP          = 'piControl'
#var_ts       = 'max_vals_eastward_wind_'
#nvar_ts      = 'max_wind'
var_ts       = 'max_lats_eastward_wind_'
nvar_ts      = 'max_lats'
#var_field    = 'tos'
var_field    = 'zos'
season_ts    = 'DJF'
season_field = 'MAM'
lag          = [-3,-2,-1,0,1,2,3,4,5]

# Directory/file details:
dataBen = '/gws/nopw/j04/acsis/bjharvey/storylines/data/JetDiags/'
BenType = '0to60W_fromseasmean'
datadir = '/gws/nopw/j04/acsis/jmecking/CMIP6/'

# Output file:
outfile = (datadir + 'composites/' + EXP + '_' + nvar_ts + '-' + season_ts + '_' + var_field + '-' + season_field +'_std.nc')

In [12]:
# Find all regridded files of variable:
Files = glob.glob(datadir + '/*/' + EXP + '/' + var_field + '*-EN4_' + season_field + '.nc' )
nf    = len(Files)
print(nf)

# Get lon/lat info:
ncid  = Dataset(Files[0],'r')
lon   = ncid.variables['lon'][:]
lat   = ncid.variables['lat'][:]
ncid.close()

ni = np.size(lon)
nj = np.size(lat)
nl = np.size(lag)

# Start counter for seasons (assuming always starts with MAM)
if season_ts == 'MAM':
    sts = 0
elif season_ts == 'JJA':
    sts = 1
elif season_ts == 'SON':
    sts = 2
elif season_ts == 'DJF':
    sts = 3

39


In [13]:
# Create output file if it doesn't exist:

if not os.path.isfile(outfile):
    ncid = Dataset(outfile,'w')
    # Dimensions:
    ncid.createDimension('lon',  ni)
    ncid.createDimension('lat',  nj)
    ncid.createDimension('lag',  nl)
    ncid.createDimension('model',None)

    # Variables:
    ncid.createVariable('lon',     'f8', ('lon',))
    ncid.createVariable('lat',     'f8', ('lat',))
    ncid.createVariable('lags',    'f8', ('lag',))
    ncid.createVariable('models',  'str',('model',))
    ncid.createVariable('mean',    'f8', ('model','lat','lon',))
    ncid.createVariable('min_std0','f8', ('model','lag','lat','lon',))
    ncid.createVariable('min_std1','f8', ('model','lag','lat','lon',))
    ncid.createVariable('min_std2','f8', ('model','lag','lat','lon',))
    ncid.createVariable('max_std0','f8', ('model','lag','lat','lon',))
    ncid.createVariable('max_std1','f8', ('model','lag','lat','lon',))
    ncid.createVariable('max_std2','f8', ('model','lag','lat','lon',))

    # Data:
    ncid.variables['lon'][:] = lon
    ncid.variables['lat'][:] = lat
    ncid.variables['lags'][:] = lag

    ncid.close()

    models = []
else:
    # Find out how much has been computed:
    ncid = Dataset(outfile,'r')
    models = ncid.variables['models'][:]
    ncid.close()
    
nm = len(models)

In [14]:
# Loop through all files and add if not yet computed:

for ff in range(0,nf):
    fieldfile = Files[ff]
    Model  = Clist[fieldfile.split('_')[1]]
    ENS    = fieldfile.split('_')[3].split('-')[0]
    
    mm = (Model.name + '_' + ENS)
    
    if mm in models:
        print('already computed ' + mm)
    else:
        print(fieldfile + ' being computed')
        # Read in jet index:
        if Model.name == 'MPI-ESM1-2-HR':
            infile = (dataBen + 'CMIP_' + Model.getInstitute()[2] + '_' +  Model.name + '_' + EXP + '_' + ENS + '_' + BenType + '.nc')
        else:
            infile = (dataBen + 'CMIP_' + Model.getInstitute()[0] + '_' +  Model.name + '_' + EXP + '_' + ENS + '_' + BenType + '.nc')
        ncid = Dataset(infile,'r')
        years_ts = ncid.variables['season_year'][sts::4]  # Assume starts with MAM
        ind_ts   = ncid.variables[var_ts][sts::4,0] # 0 = 850 hPa
        ncid.close()
        
        nt = np.size(years_ts)
        
        ncid  = Dataset(fieldfile,'r')
        years = ncid.variables['year'][:]
        ncid.close()
        
        
        # Compute means:
        inds = range(0,np.size(years_ts))
        data_mean = compute_mean_2D(inds)
        n_mean = np.size(inds)

        data_min_0 = np.zeros((nl,nj,ni),'float')
        data_max_0 = np.zeros((nl,nj,ni),'float')
        indl0 = np.squeeze(np.where(ind_ts < np.mean(ind_ts)))
        indh0 = np.squeeze(np.where(ind_ts > np.mean(ind_ts)))
        n_min0 = np.zeros((nl),'float')
        n_max0 = np.zeros((nl),'float')

        data_min_1 = np.zeros((nl,nj,ni),'float')
        data_max_1 = np.zeros((nl,nj,ni),'float')
        indl1 = np.squeeze(np.where(ind_ts < np.mean(ind_ts)-np.std(ind_ts)))
        indh1 = np.squeeze(np.where(ind_ts > np.mean(ind_ts)+np.std(ind_ts)))
        n_min1 = np.zeros((nl),'float')
        n_max1 = np.zeros((nl),'float')

        data_min_2 = np.zeros((nl,nj,ni),'float')
        data_max_2 = np.zeros((nl,nj,ni),'float')
        indl2 = np.squeeze(np.where(ind_ts < np.mean(ind_ts)-2*np.std(ind_ts)))
        indh2 = np.squeeze(np.where(ind_ts > np.mean(ind_ts)+2*np.std(ind_ts)))
        if np.size(indl2) == 1:
            indl2 = np.asarray([indl2,])
        if np.size(indh2) == 1:
            indh2 = np.asarray([indh2,])
        n_min2 = np.zeros((nl),'float')
        n_max2 = np.zeros((nl),'float')

        for ll in range(0,nl):
            # Low values:
            inds  = indl0 + lag[ll]
            inds2 = np.where((inds >= 0) & (inds < nt))
            inds = inds[inds2]
            data_min_0[ll,:,:] = compute_mean_2D(inds)
            n_min0[ll] = np.size(inds)

            inds  = indl1 + lag[ll]
            inds2 = np.where((inds >= 0) & (inds < nt))
            inds = inds[inds2]
            data_min_1[ll,:,:] = compute_mean_2D(inds)
            n_min1[ll] = np.size(inds)

            inds  = indl2 + lag[ll]
            inds2 = np.where((inds >= 0) & (inds < nt))
            inds = inds[inds2]
            data_min_2[ll,:,:] = compute_mean_2D(inds)
            n_min2[ll] = np.size(inds)

            # High values:
            inds  = indh0 + lag[ll]
            inds2 = np.where((inds >= 0) & (inds < nt))
            inds = inds[inds2]
            data_max_0[ll,:,:] = compute_mean_2D(inds)
            n_max0[ll] = np.size(inds)

            inds  = indh1 + lag[ll]
            inds2 = np.where((inds >= 0) & (inds < nt))
            inds = inds[inds2]
            data_max_1[ll,:,:] = compute_mean_2D(inds)
            n_max1[ll] = np.size(inds)

            inds  = indh2 + lag[ll]
            inds2 = np.where((inds >= 0) & (inds < nt))
            inds = inds[inds2]
            data_max_2[ll,:,:] = compute_mean_2D(inds)
            n_max2[ll] = np.size(inds)
            
        # Write to file:
        ncid = Dataset(outfile,'a')
        ncid.variables['models'  ][nm]       = mm
        ncid.variables['mean'    ][nm,:,:]   = data_mean
        ncid.variables['min_std0'][nm,:,:,:] = data_min_0
        ncid.variables['min_std1'][nm,:,:,:] = data_min_1
        ncid.variables['min_std2'][nm,:,:,:] = data_min_2
        ncid.variables['max_std0'][nm,:,:,:] = data_max_0
        ncid.variables['max_std1'][nm,:,:,:] = data_max_1
        ncid.variables['max_std2'][nm,:,:,:] = data_max_2
        ncid.close()
                                   
        nm = nm + 1

/gws/nopw/j04/acsis/jmecking/CMIP6/ACCESS-CM2/piControl/zos_ACCESS-CM2_piControl_r1i1p1f1-EN4_MAM.nc being computed


/home/users/jmecking001/miniconda3/envs/working/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


/gws/nopw/j04/acsis/jmecking/CMIP6/ACCESS-ESM1-5/piControl/zos_ACCESS-ESM1-5_piControl_r1i1p1f1-EN4_MAM.nc being computed
/gws/nopw/j04/acsis/jmecking/CMIP6/CAMS-CSM1-0/piControl/zos_CAMS-CSM1-0_piControl_r1i1p1f1-EN4_MAM.nc being computed
/gws/nopw/j04/acsis/jmecking/CMIP6/CAS-ESM2-0/piControl/zos_CAS-ESM2-0_piControl_r1i1p1f1-EN4_MAM.nc being computed
/gws/nopw/j04/acsis/jmecking/CMIP6/CESM2/piControl/zos_CESM2_piControl_r1i1p1f1-EN4_MAM.nc being computed
/gws/nopw/j04/acsis/jmecking/CMIP6/CESM2-FV2/piControl/zos_CESM2-FV2_piControl_r1i1p1f1-EN4_MAM.nc being computed
/gws/nopw/j04/acsis/jmecking/CMIP6/CESM2-WACCM/piControl/zos_CESM2-WACCM_piControl_r1i1p1f1-EN4_MAM.nc being computed
/gws/nopw/j04/acsis/jmecking/CMIP6/CESM2-WACCM-FV2/piControl/zos_CESM2-WACCM-FV2_piControl_r1i1p1f1-EN4_MAM.nc being computed
/gws/nopw/j04/acsis/jmecking/CMIP6/CIESM/piControl/zos_CIESM_piControl_r1i1p1f1-EN4_MAM.nc being computed
/gws/nopw/j04/acsis/jmecking/CMIP6/CMCC-CM2-SR5/piControl/zos_CMCC-CM2-SR5